In [ ]:
# Spanish Voice Cloning with Tortoise TTS

<a href="https://colab.research.google.com/github/juanvolpe/voiceJuan/blob/main/colab_spanish_tts.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook will help you:
1. Set up the Spanish voice cloning system
2. Upload your voice samples
3. Generate Spanish speech with your voice

Let us begin with the setup:


In [ ]:
# Check version and setup
import requests

def get_latest_commit():
    repo_api = "https://api.github.com/repos/juanvolpe/voiceJuan/commits/main"
    response = requests.get(repo_api)
    if response.status_code == 200:
        return response.json()['sha']
    return None

# Clone repository
!git clone https://github.com/juanvolpe/voiceJuan.git
%cd voiceJuan

# Check version
latest_commit = get_latest_commit()
if latest_commit:
    print(f"\nLatest GitHub commit: {latest_commit}")
    print("\nIf you want the latest version:")
    print("1. File → Reset runtime")
    print("2. Runtime → Run all")

# Install dependencies
!pip install -r requirements.txt


In [ ]:
# Spanish Voice Cloning with Tortoise TTS

<a href="https://colab.research.google.com/github/juanvolpe/voiceJuan/blob/main/colab_spanish_tts.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook will help you:
1. Set up the Spanish voice cloning system
2. Upload your voice samples
3. Generate Spanish speech with your voice

Let us begin with the setup:


In [ ]:
# Check version and setup
import requests

def get_latest_commit():
    repo_api = "https://api.github.com/repos/juanvolpe/voiceJuan/commits/main"
    response = requests.get(repo_api)
    if response.status_code == 200:
        return response.json()['sha']
    return None

# Clone repository
!git clone https://github.com/juanvolpe/voiceJuan.git
%cd voiceJuan

# Check version
latest_commit = get_latest_commit()
if latest_commit:
    print(f"\nLatest GitHub commit: {latest_commit}")
    print("\nIf you want the latest version:")
    print("1. File → Reset runtime")
    print("2. Runtime → Run all")

# Install dependencies
!pip install -r requirements.txt


In [ ]:
## Upload Voice Samples

Please prepare your WAV files with these requirements:
- Clear Spanish speech
- WAV format
- Good quality audio

Use the "Choose Files" button below to upload your samples:


In [ ]:
from google.colab import files
import os

# Create directories
!mkdir -p tortoise/voices/juan_es/samples

# Upload interface
print("Please upload your WAV files...")
uploaded = files.upload()

# Save files
for filename in uploaded.keys():
    if filename.endswith('.wav'):
        path = f'tortoise/voices/juan_es/samples/{filename}'
        with open(path, 'wb') as f:
            f.write(uploaded[filename])
        print(f'Saved {filename}')

# List all uploaded samples
print("\nUploaded voice samples:")
!ls tortoise/voices/juan_es/samples/


In [ ]:
## Generate Speech

Ready to generate speech with your voice samples! You will have two options:
1. Use existing voice cache (faster)
2. Reprocess voice samples (choose this if you added new samples)

Run the code below to begin:


In [ ]:
from spanish_tortoise import SpanishTTS
from IPython.display import Audio

# Initialize TTS
tts = SpanishTTS()  # Will ask about cache usage

# Get text input
text = input("Enter Spanish text: ")

# Available presets
presets = ['ultra_fast', 'fast', 'standard', 'high_quality']
print("\nAvailable quality presets:")
for i, p in enumerate(presets, 1):
    print(f"{i}. {p}")

# Get preset choice
while True:
    choice = input("\nSelect quality (1-4) [default=2]: ").strip()
    if not choice:
        preset = 'fast'
        break
    try:
        idx = int(choice) - 1
        if 0 <= idx < len(presets):
            preset = presets[idx]
            break
    except ValueError:
        pass
    print("Please enter a number between 1 and 4")

# Generate speech
output_file = tts.generate_speech(text, preset=preset)

# Play the generated audio
Audio(output_file)


In [ ]:
## Download Generated Audio

Click below to save the generated audio file to your computer:


In [ ]:
files.download(output_file)


In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "name": "Spanish Voice Cloning with Tortoise TTS",
      "provenance": [],
      "collapsed_sections": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "title",
        "colab_type": "text"
      },
      "source": [
        "# Spanish Voice Cloning with Tortoise TTS\n",
        "\n",
        "<a href=\"https://colab.research.google.com/github/juanvolpe/voiceJuan/blob/main/colab_spanish_tts.ipynb\" target=\"_parent\">\n",
        "  <img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/>\n",
        "</a>\n",
        "\n",
        "This notebook will help you:\n",
        "1. Set up the Spanish voice cloning system\n",
        "2. Upload your voice samples\n",
        "3. Generate Spanish speech with your voice\n",
        "\n",
        "Let us begin with the setup:"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "setup"
      },
      "source": [
        "# Check version and setup\n",
        "import requests\n",
        "\n",
        "def get_latest_commit():\n",
        "    repo_api = \"https://api.github.com/repos/juanvolpe/voiceJuan/commits/main\"\n",
        "    response = requests.get(repo_api)\n",
        "    if response.status_code == 200:\n",
        "        return response.json()['sha']\n",
        "    return None\n",
        "\n",
        "# Clone repository\n",
        "!git clone https://github.com/juanvolpe/voiceJuan.git\n",
        "%cd voiceJuan\n",
        "\n",
        "# Check version\n",
        "latest_commit = get_latest_commit()\n",
        "if latest_commit:\n",
        "    print(f\"\\nLatest GitHub commit: {latest_commit}\")\n",
        "    print(\"\\nIf you want the latest version:\")\n",
        "    print(\"1. File → Reset runtime\")\n",
        "    print(\"2. Runtime → Run all\")\n",
        "\n",
        "# Install dependencies\n",
        "!pip install -r requirements.txt"
      ],
      "execution_count": null,
      "outputs": []
    }
  ]
}


In [ ]:
# Spanish Voice Cloning with Tortoise TTS

<a href="https://colab.research.google.com/github/juanvolpe/voiceJuan/blob/main/colab_spanish_tts.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook will help you:
1. Set up the Spanish voice cloning system
2. Upload your voice samples
3. Generate Spanish speech with your voice

Let us begin with the setup:


In [ ]:
# Check version and setup
import requests

def get_latest_commit():
    repo_api = "https://api.github.com/repos/juanvolpe/voiceJuan/commits/main"
    response = requests.get(repo_api)
    if response.status_code == 200:
        return response.json()['sha']
    return None

# Clone repository
!git clone https://github.com/juanvolpe/voiceJuan.git
%cd voiceJuan

# Check version
latest_commit = get_latest_commit()
if latest_commit:
    print(f"\nLatest GitHub commit: {latest_commit}")
    print("\nIf you want the latest version:")
    print("1. File → Reset runtime")
    print("2. Runtime → Run all")

# Install dependencies
!pip install -r requirements.txt


In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "name": "Spanish Voice Cloning with Tortoise TTS",
      "provenance": [],
      "collapsed_sections": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "title",
        "colab_type": "text"
      },
      "source": [
        "# Spanish Voice Cloning with Tortoise TTS\n",
        "\n",
        "<a href=\"https://colab.research.google.com/github/juanvolpe/voiceJuan/blob/main/colab_spanish_tts.ipynb\" target=\"_parent\">\n",
        "  <img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/>\n",
        "</a>\n",
        "\n",
        "This notebook will help you:\n",
        "1. Set up the Spanish voice cloning system\n",
        "2. Upload your voice samples\n",
        "3. Generate Spanish speech with your voice\n",
        "\n",
        "Let us begin with the setup:"
      ]
    }
  ]
}


In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "name": "Spanish Voice Cloning with Tortoise TTS",
      "provenance": [],
      "collapsed_sections": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "title",
        "colab_type": "text"
      },
      "source": [
        "# Spanish Voice Cloning with Tortoise TTS\n",
        "\n",
        "<a href=\"https://colab.research.google.com/github/juanvolpe/voiceJuan/blob/main/colab_spanish_tts.ipynb\" target=\"_parent\"><img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/></a>\n",
        "\n",
        "This notebook will help you:\n",
        "1. Set up the Spanish voice cloning system\n",
        "2. Upload your voice samples\n",
        "3. Generate Spanish speech with your voice\n",
        "\n",
        "Let us begin with the setup:"
      ]
    }
  ]
}


In [ ]:
{
  "cell_type": "code",
  "metadata": {
    "id": "setup"
  },
  "source": [
    "# Check version and setup\n",
    "import requests\n",
    "\n",
    "def get_latest_commit():\n",
    "    repo_api = \"https://api.github.com/repos/juanvolpe/voiceJuan/commits/main\"\n",
    "    response = requests.get(repo_api)\n",
    "    if response.status_code == 200:\n",
    "        return response.json()['sha']\n",
    "    return None\n",
    "\n",
    "# Clone repository\n",
    "!git clone https://github.com/juanvolpe/voiceJuan.git\n",
    "%cd voiceJuan\n",
    "\n",
    "# Check version\n",
    "latest_commit = get_latest_commit()\n",
    "if latest_commit:\n",
    "    print(f\"\\nLatest GitHub commit: {latest_commit}\")\n",
    "    print(\"\\nIf you want the latest version:\")\n",
    "    print(\"1. File → Reset runtime\")\n",
    "    print(\"2. Runtime → Run all\")\n",
    "\n",
    "# Install dependencies\n",
    "!pip install -r requirements.txt"
  ],
  "execution_count": null,
  "outputs": []
}


In [ ]:
# Spanish Voice Cloning with Tortoise TTS

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juanvolpe/voiceJuan/blob/main/colab_spanish_tts.ipynb)

This notebook will help you:
1. Set up the Spanish voice cloning system
2. Upload your voice samples
3. Generate Spanish speech with your voice

Let us begin with the setup:


In [ ]:
# Check version and setup
import requests

def get_latest_commit():
    repo_api = "https://api.github.com/repos/juanvolpe/voiceJuan/commits/main"
    response = requests.get(repo_api)
    if response.status_code == 200:
        return response.json()['sha']
    return None

# Clone repository
!git clone https://github.com/juanvolpe/voiceJuan.git
%cd voiceJuan

# Check version
latest_commit = get_latest_commit()
if latest_commit:
    print(f"\nLatest GitHub commit: {latest_commit}")
    print("\nIf you want the latest version:")
    print("1. File → Reset runtime")
    print("2. Runtime → Run all")

# Install dependencies
!pip install -r requirements.txt


In [ ]:
## Upload Voice Samples

Please prepare your WAV files with these requirements:
- Clear Spanish speech
- WAV format
- Good quality audio

Use the "Choose Files" button below to upload your samples:


In [ ]:
from google.colab import files
import os

# Create directories
!mkdir -p tortoise/voices/juan_es/samples

# Upload interface
print("Please upload your WAV files...")
uploaded = files.upload()

# Save files
for filename in uploaded.keys():
    if filename.endswith('.wav'):
        path = f'tortoise/voices/juan_es/samples/{filename}'
        with open(path, 'wb') as f:
            f.write(uploaded[filename])
        print(f'Saved {filename}')

# List all uploaded samples
print("\nUploaded voice samples:")
!ls tortoise/voices/juan_es/samples/


In [ ]:
## Generate Speech

Ready to generate speech with your voice samples! You will have two options:
1. Use existing voice cache (faster)
2. Reprocess voice samples (choose this if you added new samples)

Run the code below to begin:


In [ ]:
from spanish_tortoise import SpanishTTS
from IPython.display import Audio

# Initialize TTS
tts = SpanishTTS()  # Will ask about cache usage

# Get text input
text = input("Enter Spanish text: ")

# Available presets
presets = ['ultra_fast', 'fast', 'standard', 'high_quality']
print("\nAvailable quality presets:")
for i, p in enumerate(presets, 1):
    print(f"{i}. {p}")

# Get preset choice
while True:
    choice = input("\nSelect quality (1-4) [default=2]: ").strip()
    if not choice:
        preset = 'fast'
        break
    try:
        idx = int(choice) - 1
        if 0 <= idx < len(presets):
            preset = presets[idx]
            break
    except ValueError:
        pass
    print("Please enter a number between 1 and 4")

# Generate speech
output_file = tts.generate_speech(text, preset=preset)

# Play the generated audio
Audio(output_file)


In [ ]:
## Download Generated Audio

Click below to save the generated audio file to your computer:


In [ ]:
# Spanish Voice Cloning with Tortoise TTS

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juanvolpe/voiceJuan/blob/main/colab_spanish_tts.ipynb)

This notebook helps you:
1. Set up the Spanish voice cloning system
2. Upload your voice samples
3. Generate Spanish speech with your voice

Let us begin with the setup:


In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "name": "Spanish Voice Cloning with Tortoise TTS",
      "provenance": [],
      "collapsed_sections": [],
      "toc_visible": true
    },
    "kernelspec": {
      "display_name": "Python 3",
      "name": "python3"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "intro",
        "colab_type": "text"
      },
      "source": [
        "# Spanish Voice Cloning with Tortoise TTS\n",
        "\n",
        "[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juanvolpe/voiceJuan/blob/main/colab_spanish_tts.ipynb)\n",
        "\n",
        "This notebook will help you:\n",
        "1. Set up the Spanish voice cloning system\n",
        "2. Upload your voice samples\n",
        "3. Generate Spanish speech with your voice\n",
        "\n",
        "Let us begin with the setup:"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "setup"
      },
      "source": [
        "# Check version and setup\n",
        "import requests\n",
        "\n",
        "def get_latest_commit():\n",
        "    repo_api = \"https://api.github.com/repos/juanvolpe/voiceJuan/commits/main\"\n",
        "    response = requests.get(repo_api)\n",
        "    if response.status_code == 200:\n",
        "        return response.json()['sha']\n",
        "    return None\n",
        "\n",
        "# Clone repository\n",
        "!git clone https://github.com/juanvolpe/voiceJuan.git\n",
        "%cd voiceJuan\n",
        "\n",
        "# Check version\n",
        "latest_commit = get_latest_commit()\n",
        "if latest_commit:\n",
        "    print(f\"\\nLatest GitHub commit: {latest_commit}\")\n",
        "    print(\"\\nIf you want the latest version:\")\n",
        "    print(\"1. File → Reset runtime\")\n",
        "    print(\"2. Runtime → Run all\")\n",
        "\n",
        "# Install dependencies\n",
        "!pip install -r requirements.txt"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "upload_intro",
        "colab_type": "text"
      },
      "source": [
        "## Upload Voice Samples\n",
        "\n",
        "Please prepare your WAV files with these requirements:\n",
        "- Clear Spanish speech\n",
        "- WAV format\n",
        "- Good quality audio\n",
        "\n",
        "Use the \"Choose Files\" button below to upload your samples:"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "upload"
      },
      "source": [
        "from google.colab import files\n",
        "import os\n",
        "\n",
        "# Create directories\n",
        "!mkdir -p tortoise/voices/juan_es/samples\n",
        "\n",
        "# Upload interface\n",
        "print(\"Please upload your WAV files...\")\n",
        "uploaded = files.upload()\n",
        "\n",
        "# Save files\n",
        "for filename in uploaded.keys():\n",
        "    if filename.endswith('.wav'):\n",
        "        path = f'tortoise/voices/juan_es/samples/{filename}'\n",
        "        with open(path, 'wb') as f:\n",
        "            f.write(uploaded[filename])\n",
        "        print(f'Saved {filename}')\n",
        "\n",
        "# List all uploaded samples\n",
        "print(\"\\nUploaded voice samples:\")\n",
        "!ls tortoise/voices/juan_es/samples/"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "generate_intro",
        "colab_type": "text"
      },
      "source": [
        "## Generate Speech\n",
        "\n",
        "Ready to generate speech with your voice samples! You will have two options:\n",
        "1. Use existing voice cache (faster)\n",
        "2. Reprocess voice samples (choose this if you added new samples)\n",
        "\n",
        "Run the code below to begin:"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "generate"
      },
      "source": [
        "from spanish_tortoise import SpanishTTS\n",
        "from IPython.display import Audio\n",
        "\n",
        "# Initialize TTS\n",
        "tts = SpanishTTS()  # Will ask about cache usage\n",
        "\n",
        "# Get text input\n",
        "text = input(\"Enter Spanish text: \")\n",
        "\n",
        "# Available presets\n",
        "presets = ['ultra_fast', 'fast', 'standard', 'high_quality']\n",
        "print(\"\\nAvailable quality presets:\")\n",
        "for i, p in enumerate(presets, 1):\n",
        "    print(f\"{i}. {p}\")\n",
        "\n",
        "# Get preset choice\n",
        "while True:\n",
        "    choice = input(\"\\nSelect quality (1-4) [default=2]: \").strip()\n",
        "    if not choice:\n",
        "        preset = 'fast'\n",
        "        break\n",
        "    try:\n",
        "        idx = int(choice) - 1\n",
        "        if 0 <= idx < len(presets):\n",
        "            preset = presets[idx]\n",
        "            break\n",
        "    except ValueError:\n",
        "        pass\n",
        "    print(\"Please enter a number between 1 and 4\")\n",
        "\n",
        "# Generate speech\n",
        "output_file = tts.generate_speech(text, preset=preset)\n",
        "\n",
        "# Play the generated audio\n",
        "Audio(output_file)"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "download_intro",
        "colab_type": "text"
      },
      "source": [
        "## Download Generated Audio\n",
        "\n",
        "Click below to save the generated audio file to your computer:"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "download"
      },
      "source": [
        "files.download(output_file)"
      ],
      "execution_count": null,
      "outputs": []
    }
  ]
}


In [ ]:
{
  "cell_type": "markdown",
  "metadata": {
    "id": "intro",
    "colab_type": "text"
  },
  "source": [
    "# Spanish Voice Cloning with Tortoise TTS\n",
    "\n",
    "[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juanvolpe/voiceJuan/blob/main/colab_spanish_tts.ipynb)\n",
    "\n",
    "This notebook will help you:\n",
    "1. Set up the Spanish voice cloning system\n",
    "2. Upload your voice samples\n",
    "3. Generate Spanish speech with your voice\n",
    "\n",
    "Let us begin with the setup:"
  ]
}


In [ ]:
{
  "cell_type": "markdown",
  "metadata": {
    "id": "upload_section"
  },
  "source": [
    "## Upload Voice Samples\n",
    "\n",
    "Please prepare your WAV files with these requirements:\n",
    "- Clear Spanish speech\n",
    "- WAV format\n",
    "- Good quality audio\n",
    "\n",
    "Use the \"Choose Files\" button below to upload your samples:"
  ]
}


In [ ]:
{
  "cell_type": "code",
  "metadata": {
    "id": "upload_code"
  },
  "source": [
    "from google.colab import files\n",
    "import os\n",
    "\n",
    "# Create directories\n",
    "!mkdir -p tortoise/voices/juan_es/samples\n",
    "\n",
    "# Upload interface\n",
    "print(\"Please upload your WAV files...\")\n",
    "uploaded = files.upload()\n",
    "\n",
    "# Save files\n",
    "for filename in uploaded.keys():\n",
    "    if filename.endswith('.wav'):\n",
    "        path = f'tortoise/voices/juan_es/samples/{filename}'\n",
    "        with open(path, 'wb') as f:\n",
    "            f.write(uploaded[filename])\n",
    "        print(f'Saved {filename}')\n",
    "\n",
    "# List all uploaded samples\n",
    "print(\"\\nUploaded voice samples:\")\n",
    "!ls tortoise/voices/juan_es/samples/"
  ],
  "execution_count": null,
  "outputs": []
}


In [ ]:
# Spanish Voice Cloning with Tortoise TTS

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juanvolpe/voiceJuan/blob/main/colab_spanish_tts.ipynb)

This notebook will help you:
1. Set up the Spanish voice cloning system
2. Upload your voice samples
3. Generate Spanish speech with your voice

Let us begin with the setup:


In [ ]:
{
 "cell_type": "markdown",
 "metadata": {
   "id": "upload_intro"
 },
 "source": [
   "## Upload Voice Samples\n",
   "\n",
   "Please prepare your WAV files with these requirements:\n",
   "- Clear Spanish speech\n",
   "- WAV format\n",
   "- Good quality audio\n",
   "\n",
   "Use the \"Choose Files\" button below to upload your samples:"
 ]
}


In [ ]:
# Check which commit we're using
!git clone https://github.com/juanvolpe/voiceJuan.git
%cd voiceJuan
!git rev-parse HEAD
print("\nIf this matches 86c3578751c2981a77e09cb2d61fe4179deb9f92, you're on the latest version!")

# Install dependencies
!pip install -r requirements.txt


In [ ]:
{
 "cell_type": "markdown",
 "metadata": {
   "id": "generate_intro"
 },
 "source": [
   "## Generate Speech\n",
   "\n",
   "Ready to generate speech with your voice samples! You will have two options:\n",
   "1. Use existing voice cache (faster)\n",
   "2. Reprocess voice samples (choose this if you added new samples)\n",
   "\n",
   "Run the code below to begin:"
 ]
}


In [ ]:
# Spanish Voice Cloning with Tortoise TTS

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juanvolpe/voiceJuan/blob/main/colab_spanish_tts.ipynb)

This notebook will help you:
1. Set up the Spanish voice cloning system
2. Upload your voice samples
3. Generate Spanish speech with your voice

Let us set up the environment:


In [ ]:
{
 "cell_type": "markdown",
 "metadata": {
   "id": "download_intro"
 },
 "source": [
   "## Download Generated Audio\n",
   "\n",
   "Click below to save the generated audio file to your computer:"
 ]
}


In [ ]:
## Upload Voice Samples

Please prepare your WAV files with these requirements:
- Clear Spanish speech
- WAV format
- Good quality audio

Use the "Choose Files" button below to upload your samples:


In [ ]:
# Spanish Voice Cloning with Tortoise TTS

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juanvolpe/voiceJuan/blob/main/colab_spanish_tts.ipynb)

This notebook will help you:
1. Set up the Spanish voice cloning system
2. Upload your voice samples
3. Generate Spanish speech with your voice

First, let's set up the environment:


In [ ]:
## Generate Speech

Ready to generate speech with your voice samples! You will have two options:
1. Use existing voice cache (faster)
2. Reprocess voice samples (choose this if you added new samples)

Run the code below to begin:


In [ ]:
# Clone the repository and install dependencies
!git clone https://github.com/juanvolpe/voiceJuan.git
%cd voiceJuan
!pip install -r requirements.txt


In [ ]:
## Download Generated Audio

Click below to save the generated audio file to your computer:


In [ ]:
## Upload Voice Samples

Now you can upload your WAV files. Make sure they are:
- Clear Spanish speech
- WAV format
- Good quality audio

Run the cell below and use the 'Choose Files' button to upload your samples:


In [ ]:
from google.colab import files
import os

# Create directories
!mkdir -p tortoise/voices/juan_es/samples

# Upload interface
print("Please upload your WAV files...")
uploaded = files.upload()

# Save files
for filename in uploaded.keys():
    if filename.endswith('.wav'):
        path = f'tortoise/voices/juan_es/samples/{filename}'
        with open(path, 'wb') as f:
            f.write(uploaded[filename])
        print(f'Saved {filename}')

# List all uploaded samples
print("\nUploaded voice samples:")
!ls tortoise/voices/juan_es/samples/


In [ ]:
## Generate Speech

Now you can generate speech using your voice samples. The system will ask if you want to:
1. Use existing voice cache (faster)
2. Reprocess voice samples (use if you've added new samples)

Run the cell below to start generating:


In [ ]:
from spanish_tortoise import SpanishTTS
from IPython.display import Audio

# Initialize TTS
tts = SpanishTTS()  # Will ask about cache usage

# Get text input
text = input("Enter Spanish text: ")

# Available presets
presets = ['ultra_fast', 'fast', 'standard', 'high_quality']
print("\nAvailable quality presets:")
for i, p in enumerate(presets, 1):
    print(f"{i}. {p}")

# Get preset choice
while True:
    choice = input("\nSelect quality (1-4) [default=2]: ").strip()
    if not choice:
        preset = 'fast'
        break
    try:
        idx = int(choice) - 1
        if 0 <= idx < len(presets):
            preset = presets[idx]
            break
    except ValueError:
        pass
    print("Please enter a number between 1 and 4")

# Generate speech
output_file = tts.generate_speech(text, preset=preset)

# Play the generated audio
Audio(output_file)


In [ ]:
## Download Generated Audio

You can download the generated audio file:


In [ ]:
files.download(output_file)


In [ ]:
# Spanish Voice Cloning with Tortoise TTS

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juanvolpe/voiceJuan/blob/main/colab_spanish_tts.ipynb)

This notebook will help you:
1. Set up the Spanish voice cloning system
2. Upload your voice samples
3. Generate Spanish speech with your voice

First, let's set up the environment:


In [ ]:
# Clone the repository
!git clone https://github.com/juanvolpe/voiceJuan.git
%cd voiceJuan

# Install dependencies
!pip install -r requirements.txt


In [ ]:
## Upload Voice Samples

Now you can upload your WAV files. Make sure they are:
- Clear Spanish speech
- WAV format
- Good quality audio

Run the cell below and use the 'Choose Files' button to upload your samples:


In [ ]:
from google.colab import files
import os

# Create directories
!mkdir -p tortoise/voices/juan_es/samples

# Upload interface
print("Please upload your WAV files...")
uploaded = files.upload()

# Save files
for filename in uploaded.keys():
    if filename.endswith('.wav'):
        path = f'tortoise/voices/juan_es/samples/{filename}'
        with open(path, 'wb') as f:
            f.write(uploaded[filename])
        print(f'Saved {filename}')

# List all uploaded samples
print("\nUploaded voice samples:")
!ls tortoise/voices/juan_es/samples/


In [ ]:
## Generate Speech

Now you can generate speech using your voice samples. The system will ask if you want to:
1. Use existing voice cache (faster)
2. Reprocess voice samples (use if you've added new samples)

Run the cell below to start generating:


In [ ]:
from spanish_tortoise import SpanishTTS
from IPython.display import Audio

# Initialize TTS
tts = SpanishTTS()  # Will ask about cache usage

# Get text input
text = input("Enter Spanish text: ")

# Available presets
presets = ['ultra_fast', 'fast', 'standard', 'high_quality']
print("\nAvailable quality presets:")
for i, p in enumerate(presets, 1):
    print(f"{i}. {p}")

# Get preset choice
while True:
    choice = input("\nSelect quality (1-4) [default=2]: ").strip()
    if not choice:
        preset = 'fast'
        break
    try:
        idx = int(choice) - 1
        if 0 <= idx < len(presets):
            preset = presets[idx]
            break
    except ValueError:
        pass
    print("Please enter a number between 1 and 4")

# Generate speech
output_file = tts.generate_speech(text, preset=preset)

# Play the generated audio
Audio(output_file)


In [ ]:
## Download Generated Audio

You can download the generated audio file:


In [ ]:
files.download(output_file)
